In [1]:
import pandas as pd
import numpy as np
import empyrical as ep
from datetime import datetime, timedelta

In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
from __future__ import annotations
from pandas_market_calendars import get_calendar

In [4]:
#设计一个观察日的函数
def get_same_day_dates(start_date, n ,lock):
    dates = []
    begin_date=datetime.strptime(start_date, '%Y-%m-%d')
    for i in range(lock,n+1):
        print(i)
        current_date = begin_date
        time=i
        current_date += relativedelta(months=time)  # 将日期增加一个月
        dates.append(current_date.strftime('%Y-%m-%d'))  # 将日期对象转换为字符串格式并添加到列表中
    return dates

In [5]:
from pandas_market_calendars import get_calendar

# 获取上交所交易日历对象
sh_exchange_calendar = get_calendar('XSHG')

# 指定开始日期和结束日期
start_date = pd.to_datetime('2004-01-01')
mid_date1=pd.to_datetime('2022-09-09')
mid_date2=pd.to_datetime('2022-09-13')
end_date = pd.to_datetime('2026-12-31')

# 获取交易日日期列表
trading_days1 = sh_exchange_calendar.valid_days(start_date=start_date, end_date=mid_date1)
trading_days2 = sh_exchange_calendar.valid_days(start_date=mid_date2, end_date=end_date)

# 将交易日日期列表转换为字符串格式
trading_days_list = trading_days1.strftime('%Y-%m-%d').tolist()+trading_days2.strftime('%Y-%m-%d').tolist()
dates_to_remove = ['2022-10-06', '2022-10-07','2023-05-02','2023-05-03','2023-06-23']

for date in dates_to_remove:
    if date in trading_days_list:
        trading_days_list.remove(date)

In [6]:
#若为非交易日,则往后顺延到最近的交易日

def adjust_to_trading_days(dates, trading_days_list):
    adjusted_dates = []

    trading_days = pd.to_datetime(trading_days_list)

    for date in dates:
        dt = pd.to_datetime(date)

        if dt in trading_days:
            adjusted_dates.append(date)
        else:
            next_trading_day = trading_days[trading_days > dt].min()
            adjusted_dates.append(next_trading_day.strftime('%Y-%m-%d'))

    return adjusted_dates

In [7]:
#新撰写一个函数，作为后面计算降敲函数敲出价的辅助，date为单个日期，obs_date为日期列表，
#返回列表中早于单个日期列表的个数
def count_earlier_dates(date, obs_date):
    count = 0
    given_date = datetime.strptime(date, '%Y-%m-%d')
    
    for date_str in obs_date:
        obs = datetime.strptime(date_str, '%Y-%m-%d')
        if obs < given_date:
            count += 1
    
    return count

## 美式鲨鱼鳍结构

In [22]:
#形成最终的结果函数
#n代表产品的期限,r代表能达到的收益率上限，达到上限在下文中称为突破
#begin_date代表开始日期，end_date代表终止日期,interrupt_date代表截断日期，roll_length代表PB,PE分位数向前滚动的天数
def shark_fin(df_all,df_pbpe,n,lock,strike,r,begin_date,end_date,interrupt_date,roll_length):
    df_pbpe['PB分位数']=df_pbpe['PB'].rolling(roll_length).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    df_pbpe['PE分位数']=df_pbpe['PE'].rolling(roll_length).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    df=df_all
    df.columns=['date','标的指数收盘价']
    df.index=df['date']
    df['敲出价格'] = (1+r)*df['标的指数收盘价']
    
    #新增一列记录是否发生敲出/发生突破
    df['是否敲出']=0
    
    #新增一列记录敲出或者到期或者截断的时间
    df['敲出或者到期时间']=0
    
    #新增一列记录产品敲出时指数点位
    df['敲出或到期时指数点位']=0
    
    #新增一列填入产品收益率
    df['年化收益率']=0  
    
    #新增一列计算当前指数点位的历史分位数（滚动周期与PBPE计算周期相同）
    df['指数点位历史分位数']=df['标的指数收盘价'].rolling(roll_length).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    
    #新增一列判断产品是否存续
    df['产品当前是否存续']=0
    
    #新增一列计算指数近20天波动率以及波动率分位数
    df['过去20天指数波动率'] = df['标的指数收盘价'].rolling(window=20).std()
    df['过去20天波动率分位数']=df['过去20天指数波动率'].rolling(252).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    
    #截取需要部分
    df=df.loc[begin_date:end_date]
    df_pbpe=df_pbpe.loc[begin_date:end_date]

    #将PB/PE的数据读出并存入
    df_part1=pd.concat([df, df_pbpe], axis=1)
    print(df_part1)
    
    #选取日截至到interrupt_date
    df_return=df_part1.loc[:interrupt_date]
    print(df_return)
    
    
    #判断是否敲出并对以上信息进行填充
    for k in range(len(df_return)):
        start_date=df_return.iloc[k,0]
        dates=get_same_day_dates(start_date, n ,lock)
        obs_date_list = adjust_to_trading_days(dates,trading_days_list)
        print("obs1"+str(obs_date_list))
        obs_date=[date for date in obs_date_list if datetime.strptime(date, "%Y-%m-%d") <= datetime.strptime(end_date, "%Y-%m-%d")]
      
        if len(obs_date_list)>len(obs_date):
            df_return.iloc[k,8]= 1
          
        #提取观察日对应的标的指数点位
        price_obs_date= df_part1.loc[obs_date,'标的指数收盘价']
      
        #找到敲出的时间点
        knock_price=df_return.iloc[k,2]
        
        position = None
        for i, price in enumerate(price_obs_date):
            if price > knock_price:
                position = i
                df_return.iloc[k,3]=1
                #如果发生敲出/突破，记录产品收益率
                df_return.iloc[k,6]=r
                break

        if position is None:
            position = len(price_obs_date) - 1 
            #如果未发生敲出/突破，根据期末涨跌幅确认产品收益
            multiplier=n/12
            price_change=df_part1.loc[obs_date[position],'标的指数收盘价']/df_return.iloc[k,1]-1
            if price_change>r:
                df_return.iloc[k,6]=r
            elif price_change>0:
                df_return.iloc[k,6]=price_change/multiplier
            else:
                df_return.iloc[k,6]=0
    
        #记录敲出或者到期的时间
        df_return.iloc[k,4]=obs_date[position]
        print(obs_date[position])
        
        #记录敲出或到期时指数点位
        df_return.iloc[k,5]=df_part1.loc[obs_date[position],'标的指数收盘价']
    
    
    # 计算两个日期之间的差距
    df_return['date'] = pd.to_datetime(df_return['date'])
    df_return['敲出或者到期时间'] = pd.to_datetime(df_return['敲出或者到期时间'])
    df_return['产品存续天数'] = (df_return['突破或者到期时间'] - df_return['date']).dt.days
             
    return df_return

In [9]:
#这里有一个确定分位数滚动从何时开始的小函数
from datetime import datetime, timedelta

def find_trading_day(begin_date, trading_day_list,roll_length):
    # 将日期字符转换为datetime对象
    date_format = "%Y-%m-%d"  # 假设日期字符的格式为"年-月-日"，如"2023-07-21"
    begin_date_obj = datetime.strptime(begin_date, date_format)

    # 找到begin_date在trading_day_list中的索引
    index_of_begin_date = trading_day_list.index(begin_date)

    # 找到在trading_day_list中的第252个交易日期
    days_to_add = 252
    if index_of_begin_date + days_to_add < len(trading_day_list):
        result_date = trading_day_list[index_of_begin_date + days_to_add]
        return result_date
    else:
        return None  # 如果超出日期列表的范围，则返回None或者其他指定的值

## 先设置后面使用的一些基本参数

In [15]:
#设置标的
trade_code='000852.SH'
#设置时间
begin_date="2017-01-01"
#begin_date=find_trading_day("2004-01-02",trading_days_list,252*3)
end_date="2023-07-28"
interrupt_date="2022-07-28"
#设置产品总存续时长
n=12
#设置锁定时长
lock=3
#设置敲出界限
strike=1.10
#设置鲨鱼鳍敲出后收益率
r=0.035

## 用wind接口提取数据

In [11]:
#用wind提取数据
#导入库
from WindPy import w
import pandas as pd
w.start()
#数据提取
wdata1=w.wsd(trade_code, "close", "2013-01-01", end_date, "")
df=pd.DataFrame(wdata1.Data,columns=wdata1.Times,index=wdata1.Codes).T
df['date']=df.index
df['标的指数收盘价']=df.iloc[:,0]
df=df[['date','标的指数收盘价']]
#提取PB和PE的数据
wdata2=w.wsd(trade_code, "pe_ttm,pb_lf", "2013-01-01", end_date, "")
df_pbpe1=pd.DataFrame(wdata2.Times)
df_pbpe2=pd.DataFrame(wdata2.Data).T
df_pbpe=pd.concat([df_pbpe1, df_pbpe2], axis=1)
df_pbpe.columns=['交易日期','PE','PB']
w.close()
#将读取的数据存入文件以防止多次读取浪费提取次数
df.to_excel('df.xlsx',index=False)
df_pbpe.to_excel('df_pbpe.xlsx',index=False)

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [24]:
#在使用wind读取并存入数据后，将数据读出，以便之后运算
df=pd.read_excel('df.xlsx')
df_pbpe=pd.read_excel('df_pbpe.xlsx')
df['date']=df['date'].dt.strftime('%Y-%m-%d')
df_pbpe['交易日期']=df_pbpe['交易日期'].dt.strftime('%Y-%m-%d')
df_pbpe.index=df_pbpe['交易日期']
df_pbpe=df_pbpe[['PE','PB']]

## 调入函数并进行运算

In [25]:
df_return=shark_fin(df,df_pbpe,n,lock,strike,r,begin_date,end_date,interrupt_date,252*3)
df_return.head(10)

                  date    标的指数收盘价         敲出价格  是否敲出  敲出或者到期时间  敲出或到期时指数点位  \
2017-01-03  2017-01-03  8565.1363  8864.916070     0         0           0   
2017-01-04  2017-01-04  8669.9492  8973.397422     0         0           0   
2017-01-05  2017-01-05  8670.7612  8974.237842     0         0           0   
2017-01-06  2017-01-06  8609.8985  8911.244947     0         0           0   
2017-01-09  2017-01-09  8653.8468  8956.731438     0         0           0   
...                ...        ...          ...   ...       ...         ...   
2023-07-24  2023-07-24  6393.7938  6617.576583     0         0           0   
2023-07-25  2023-07-25  6498.2688  6725.708208     0         0           0   
2023-07-26  2023-07-26  6457.2535  6683.257372     0         0           0   
2023-07-27  2023-07-27  6407.5461  6631.810213     0         0           0   
2023-07-28  2023-07-28  6465.3763  6691.664470     0         0           0   

            年化收益率  指数点位历史分位数  产品当前是否存续  过去20天指数波动率  过去20天波动率分位数

E:\myresource\apps\anaconda\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


obs1['2017-04-19', '2017-05-19', '2017-06-19', '2017-07-19', '2017-08-21', '2017-09-19', '2017-10-19', '2017-11-20', '2017-12-19', '2018-01-19']
obs22017-04-19    2017-04-19
2017-04-20    2017-04-20
2017-04-21    2017-04-21
2017-04-24    2017-04-24
2017-04-25    2017-04-25
                 ...    
2018-01-15    2018-01-15
2018-01-16    2018-01-16
2018-01-17    2018-01-17
2018-01-18    2018-01-18
2018-01-19    2018-01-19
Name: date, Length: 189, dtype: object
2018-01-19
3
4
5
6
7
8
9
10
11
12
obs1['2017-04-20', '2017-05-22', '2017-06-20', '2017-07-20', '2017-08-21', '2017-09-20', '2017-10-20', '2017-11-20', '2017-12-20', '2018-01-22']
obs22017-04-20    2017-04-20
2017-04-21    2017-04-21
2017-04-24    2017-04-24
2017-04-25    2017-04-25
2017-04-26    2017-04-26
                 ...    
2018-01-16    2018-01-16
2018-01-17    2018-01-17
2018-01-18    2018-01-18
2018-01-19    2018-01-19
2018-01-22    2018-01-22
Name: date, Length: 189, dtype: object
2018-01-22
3
4
5
6
7
8
9
10
11
12
obs1['

obs1['2017-06-02', '2017-07-03', '2017-08-02', '2017-09-04', '2017-10-09', '2017-11-02', '2017-12-04', '2018-01-02', '2018-02-02', '2018-03-02']
obs22017-06-02    2017-06-02
2017-06-05    2017-06-05
2017-06-06    2017-06-06
2017-06-07    2017-06-07
2017-06-08    2017-06-08
                 ...    
2018-02-26    2018-02-26
2018-02-27    2018-02-27
2018-02-28    2018-02-28
2018-03-01    2018-03-01
2018-03-02    2018-03-02
Name: date, Length: 185, dtype: object
2018-03-02
3
4
5
6
7
8
9
10
11
12
obs1['2017-06-05', '2017-07-03', '2017-08-03', '2017-09-04', '2017-10-09', '2017-11-03', '2017-12-04', '2018-01-03', '2018-02-05', '2018-03-05']
obs22017-06-05    2017-06-05
2017-06-06    2017-06-06
2017-06-07    2017-06-07
2017-06-08    2017-06-08
2017-06-09    2017-06-09
                 ...    
2018-02-27    2018-02-27
2018-02-28    2018-02-28
2018-03-01    2018-03-01
2018-03-02    2018-03-02
2018-03-05    2018-03-05
Name: date, Length: 185, dtype: object
2018-03-05
3
4
5
6
7
8
9
10
11
12
obs1['

obs1['2017-07-17', '2017-08-17', '2017-09-18', '2017-10-17', '2017-11-17', '2017-12-18', '2018-01-17', '2018-02-22', '2018-03-19', '2018-04-17']
obs22017-07-17    2017-07-17
2017-07-18    2017-07-18
2017-07-19    2017-07-19
2017-07-20    2017-07-20
2017-07-21    2017-07-21
                 ...    
2018-04-11    2018-04-11
2018-04-12    2018-04-12
2018-04-13    2018-04-13
2018-04-16    2018-04-16
2018-04-17    2018-04-17
Name: date, Length: 184, dtype: object
2018-04-17
3
4
5
6
7
8
9
10
11
12
obs1['2017-07-18', '2017-08-18', '2017-09-18', '2017-10-18', '2017-11-20', '2017-12-18', '2018-01-18', '2018-02-22', '2018-03-19', '2018-04-18']
obs22017-07-18    2017-07-18
2017-07-19    2017-07-19
2017-07-20    2017-07-20
2017-07-21    2017-07-21
2017-07-24    2017-07-24
                 ...    
2018-04-12    2018-04-12
2018-04-13    2018-04-13
2018-04-16    2018-04-16
2018-04-17    2018-04-17
2018-04-18    2018-04-18
Name: date, Length: 184, dtype: object
2018-04-18
3
4
5
6
7
8
9
10
11
12
obs1['

...

Name: date, Length: 184, dtype: object
2022-07-18
3
4
5
6
7
8
9
10
11
12
obs1['2022-07-19', '2022-08-19', '2022-09-19', '2022-10-19', '2022-11-21', '2022-12-19', '2023-01-19', '2023-02-20', '2023-03-20', '2023-04-19']
obs22022-07-19    2022-07-19
2022-07-20    2022-07-20
2022-07-21    2022-07-21
2022-07-22    2022-07-22
2022-07-25    2022-07-25
                 ...    
2023-04-13    2023-04-13
2023-04-14    2023-04-14
2023-04-17    2023-04-17
2023-04-18    2023-04-18
2023-04-19    2023-04-19
Name: date, Length: 184, dtype: object
2022-07-19
3
4
5
6
7
8
9
10
11
12
obs1['2022-07-20', '2022-08-22', '2022-09-20', '2022-10-20', '2022-11-21', '2022-12-20', '2023-01-20', '2023-02-20', '2023-03-20', '2023-04-20']
obs22022-07-20    2022-07-20
2022-07-21    2022-07-21
2022-07-22    2022-07-22
2022-07-25    2022-07-25
2022-07-26    2022-07-26
                 ...    
2023-04-14    2023-04-14
2023-04-17    2023-04-17
2023-04-18    2023-04-18
2023-04-19    2023-04-19
2023-04-20    2023-04-20
Name: 

obs1['2022-08-25', '2022-09-26', '2022-10-25', '2022-11-25', '2022-12-26', '2023-01-30', '2023-02-27', '2023-03-27', '2023-04-25', '2023-05-25']
obs22022-08-25    2022-08-25
2022-08-26    2022-08-26
2022-08-29    2022-08-29
2022-08-30    2022-08-30
2022-08-31    2022-08-31
                 ...    
2023-05-19    2023-05-19
2023-05-22    2023-05-22
2023-05-23    2023-05-23
2023-05-24    2023-05-24
2023-05-25    2023-05-25
Name: date, Length: 180, dtype: object
2022-08-25
3
4
5
6
7
8
9
10
11
12
obs1['2022-08-26', '2022-09-26', '2022-10-26', '2022-11-28', '2022-12-26', '2023-01-30', '2023-02-27', '2023-03-27', '2023-04-26', '2023-05-26']
obs22022-08-26    2022-08-26
2022-08-29    2022-08-29
2022-08-30    2022-08-30
2022-08-31    2022-08-31
2022-09-01    2022-09-01
                 ...    
2023-05-22    2023-05-22
2023-05-23    2023-05-23
2023-05-24    2023-05-24
2023-05-25    2023-05-25
2023-05-26    2023-05-26
Name: date, Length: 180, dtype: object
2022-08-26
3
4
5
6
7
8
9
10
11
12
obs1['

obs1['2022-10-19', '2022-11-21', '2022-12-19', '2023-01-19', '2023-02-20', '2023-03-20', '2023-04-19', '2023-05-19', '2023-06-19', '2023-07-19']
obs22022-10-19    2022-10-19
2022-10-20    2022-10-20
2022-10-21    2022-10-21
2022-10-24    2022-10-24
2022-10-25    2022-10-25
                 ...    
2023-07-13    2023-07-13
2023-07-14    2023-07-14
2023-07-17    2023-07-17
2023-07-18    2023-07-18
2023-07-19    2023-07-19
Name: date, Length: 184, dtype: object
2023-07-19
3
4
5
6
7
8
9
10
11
12
obs1['2022-10-20', '2022-11-21', '2022-12-20', '2023-01-20', '2023-02-20', '2023-03-20', '2023-04-20', '2023-05-22', '2023-06-20', '2023-07-20']
obs22022-10-20    2022-10-20
2022-10-21    2022-10-21
2022-10-24    2022-10-24
2022-10-25    2022-10-25
2022-10-26    2022-10-26
                 ...    
2023-07-14    2023-07-14
2023-07-17    2023-07-17
2023-07-18    2023-07-18
2023-07-19    2023-07-19
2023-07-20    2023-07-20
Name: date, Length: 184, dtype: object
2023-07-20
3
4
5
6
7
8
9
10
11
12
obs1['

E:\myresource\apps\anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
E:\myresource\apps\anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyError: '突破或者到期时间'